In [1]:
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix

ratings = {
    'user_id': [1, 2, 4],
    'movie_id': [2, 3, 7],
    'rating': [4, 3, 1]
}
ratings = pd.DataFrame(ratings)

rating_matrix = ratings.pivot(index='user_id',
                              columns='movie_id',
                              values='rating').fillna(0)

full_matrix1 = np.array(rating_matrix)
print(full_matrix1)

[[4. 0. 0.]
 [0. 3. 0.]
 [0. 0. 1.]]


In [2]:
rating_matrix

movie_id,2,3,7
user_id,,,
1,4.0,0.0,0.0
2,0.0,3.0,0.0
4,0.0,0.0,1.0


In [5]:
data = np.array(ratings['rating'])
row_indices = np.array(ratings['user_id'])
col_indices = np.array(ratings['movie_id'])

rating_matrix = csr_matrix((data, (row_indices, col_indices)), dtype=int)
print(rating_matrix)

  (1, 2)	4
  (2, 3)	3
  (4, 7)	1


In [6]:
full_matrix2 = rating_matrix.toarray()
print(full_matrix2)

[[0 0 0 0 0 0 0 0]
 [0 0 4 0 0 0 0 0]
 [0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]]


In [22]:
import os
import pandas as pd

base_src = ''
ratings_20m_src = os.path.join(base_src, 'ratings-20m.csv')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(ratings_20m_src, sep=',',
                      names=r_cols, encoding='latin-1')


R_temp = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)


ValueError: Unstacked DataFrame is too big, causing int32 overflow

In [28]:
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
import os
import numpy as np 
import pandas as pd

base_src = ''
ratings_20m_src = os.path.join(base_src, 'ratings-20m.csv')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(ratings_20m_src,
                      sep=',',
                      names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)

ratings = ratings[[
    'user_id', 'movie_id', 'rating'
]].astype(int)

data = np.array(ratings['rating'])
row_indices = np.array(ratings['user_id'])
col_indices = np.array(ratings['movie_id'])

R_temp = csr_matrix((data, (row_indices, col_indices)), dtype=int)

class NEW_MF():
  def __init__(self, ratings, hyper_params):
    self.R = ratings
    # 사용자 수 (num_users) 와 아이템 수(num_items)를 받아온다
    self.num_users, self.num_items = np.shape(self.R)
    # 아래는 MF weight 조절을 위한 하이퍼파라미터이다
    # K : 잠재요인(latent factor)의 수
    self.K = hyper_params['K']
    # alpha 학습률
    self.alpha = hyper_params['alpha']
    # beta  정규화 계수
    self.beta = hyper_params['beta']
    # iterations SGD 계산을 할 때의 반복 횟수
    self.iterations = hyper_params['iterations']
    # verbose SGD 학습 과정을 중간중간에 출력할 것인지에 대한 여부
    self.verbose = hyper_params['verbose'] 


  def rmse(self):
    # self.R 에서 평점이 있는 요소의 인덱스를 가져온다 
    xs, ys = self.R.nonzero()
    # prediction 과 error를 담을 리스트 변수 초기화 
    self.predictions = []
    self.errors = [] 
    # 평점이 있는 요소 (사용자 x, 아이템 y) 각각에 대해서 아래의 코드를 실행한다
    for x, y in zip(xs, ys):
      # 사용자 x, 아이템 y 에 대해 평점 예측치를 get_prediction() 함수를 사용해서 계산한다.
      prediction = self.get_prediction(x, y)
      # 예측값을 예측값 리스트에 추가한다 
      self.predictions.append(prediction)
      # 실제값 과 예측값의 차이를 계산해서 오차값 리스트에 추가한다
      self.errors.append(self.R[x,y] - prediction)
    # 예측값 리스트와 오차값 리스트를 numpy array 형태로 변환한다.
    self.prediction = np.array(self.predictions)
    self.errors = np.array(self.errors)
    # error를 활용해서 RMSE 도출 
    return np.sqrt(np.mean(self.errors ** 2))

  def sgd(self):
    for i, j, r in self.samples:
      # 사용자 i, 아이템 j 에 대한 평저 예측치 계산
      prediction = self.get_prediction(i, j)
      # 실제 평점과 비교한 오차 계산
      e = (r-prediction)

      # 사용자 평가 경향 계산 및 업데이트
      self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
      # 아이템 평가 경향 계산 및 업데이트
      self.b_d[j] += self.alpha * (e - (self.beta * self.b_d[j]))

      # P 행렬 계산 및 업데이트
      self.P[i, :] += self.alpha * ((e * self.Q[j,:]) - (self.beta * self.P[i, :]))
      # Q 행렬 계산 및 업데이트
      self.Q[j, :] += self.alpha * ((e * self.P[i,:]) - (self.beta * self.Q[j, :]))
  
  def get_prediction(self, i, j):
    # 사용자 i, 아이템 j에 대한 평점 예측치를 앞에서 배웠던 식을 이용해서 구한다 
    prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i,:].dot(self.Q[j,].T)
    return prediction
 
  #  Test set 선정
  def set_test(self, ratings_test):
    test_set = []
    for i in range(len(ratings_test)):
      x, y, z = ratings_test.iloc[i]
      test_set.append([x, y, z])
      self.R[x, y] = 0
    self.test_set = test_set 
    return test_set

  # Test set RMSE 계산 
  def test_rmse(self):
    error = 0
    for one_set in self.test_set:
      predicted = self.get_prediction(one_set[0], one_set[1])
      # e => e^2
      error += pow(one_set[2] - predicted, 2)
    return np.sqrt(error/len(self.test_set))

  def test(self):
    self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
    self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

    self.b_u = np.zeros(self.num_users)
    self.b_d = np.zeros(self.num_items)
    self.b = np.mean(self.R[self.R.nonzero()])

    rows, columns = self.R.nonzero()
    self.samples = [(i,j,self.R[i,j]) for i, j in zip(rows, columns)]

    training_process = []
    for i in range(self.iterations):
      np.random.shuffle(self.samples)
      self.sgd()
      rmse1 = self.rmse()
      rmse2 = self.test_rmse()
      training_process.append((i+1, rmse1, rmse2))
      if self.verbose:
        if (i+1) % 10 == 0:
          print('Iteration : %d ; train RMSE = %.4f ; TEST RMSE = %.4f' % (i+1, rmse1, rmse2))
    return training_process

  def get_one_prediction(self, user_id, item_id):
    return self.get_prediction(user_id, item_id)
    
  def full_prediction(self):
    return self.b + self.b_u[:,np.newaxis] + self.b_d[np.newaxis,:] + self.P.dot(self.Q.T)


ratings_train, ratings_test = train_test_split(ratings,
                                               test_size=0.2,
                                               shuffle=True,
                                               random_state=2022)


hyper_params = {
    'K' : 30,
    'alpha' : 0.001,
    'beta' : 0.02,
    'iterations' : 100,
    'verbose': True
}


mf = NEW_MF(R_temp, hyper_params)
train_set = mf.set_test(ratings_test)
result = mf.test()
result